In [1]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from apex import amp

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

#Text Cleaning
from bs4 import BeautifulSoup
import re 

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [2]:
torch.cuda.is_available()

True

In [3]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.synchronize()

In [4]:
CONFIG = {"seed": 2021,
          "epochs": 10,
          "model_name": "unitary/toxic-bert",
          "train_batch_size": 28,
          "valid_batch_size": 56,
          "max_length": 100,
          "learning_rate": 1e-5,
          "epsilon" : 1e-6,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 5e-7,
          "T_max": 500,
          "weight_decay": 1e-5,
          "n_fold": 1,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "patience": 4,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [5]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [6]:
df_train = pd.read_csv("train.csv")
df_train.head()

,more_toxic,less_toxic
0,Banning guns and explosives doesn t make a sta...,Banning guns and explosives doesn t make a sta...
1,I m saying it hasn t been researched and yet t...,I m saying it hasn t been researched and yet t...
2,The fuck did I just read,what what the fuck did i just read
3,It s almost as if these leave voting money gru...,Leading Britain off a cliff and cheating their...
4,Bulgaria Croatia Czech Republic Denmark Hungar...,Bulgaria Croatia Czech Republic Denmark Hungar...


In [7]:
df_valid = pd.read_csv("input/jigsaw-toxic-severity-rating/validation_data.csv")
df_valid.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [8]:
'''class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return
        if self.parents[x] > self.parents[y]:
            x, y = y, x
        self.parents[x] += self.parents[y]
        self.parents[y] = x


def get_group_unionfind(train: pd.DataFrame):
    less_unique_text = train['less_toxic'].unique()
    more_unique_text = train['more_toxic'].unique()
    unique_text = np.hstack([less_unique_text, more_unique_text])
    unique_text = np.unique(unique_text).tolist()    
    text2num = {text: i for i, text in enumerate(unique_text)}
    num2text = {num: text for text, num in text2num.items()}
    train['num_less_toxic'] = train['less_toxic'].map(text2num)
    train['num_more_toxic'] = train['more_toxic'].map(text2num)

    uf = UnionFind(len(unique_text))
    for seq1, seq2 in train[['num_less_toxic', 'num_more_toxic']].to_numpy():
        uf.union(seq1, seq2)

    text2group = {num2text[i]: uf.find(i) for i in range(len(unique_text))}
    train['group'] = train['less_toxic'].map(text2group)
    train = train.drop(columns=['num_less_toxic', 'num_more_toxic'])
    return train'''

"class UnionFind():\n    def __init__(self, n):\n        self.n = n\n        self.parents = [-1] * n\n\n    def find(self, x):\n        if self.parents[x] < 0:\n            return x\n        else:\n            self.parents[x] = self.find(self.parents[x])\n            return self.parents[x]\n\n    def union(self, x, y):\n        x = self.find(x)\n        y = self.find(y)\n        if x == y:\n            return\n        if self.parents[x] > self.parents[y]:\n            x, y = y, x\n        self.parents[x] += self.parents[y]\n        self.parents[y] = x\n\n\ndef get_group_unionfind(train: pd.DataFrame):\n    less_unique_text = train['less_toxic'].unique()\n    more_unique_text = train['more_toxic'].unique()\n    unique_text = np.hstack([less_unique_text, more_unique_text])\n    unique_text = np.unique(unique_text).tolist()    \n    text2num = {text: i for i, text in enumerate(unique_text)}\n    num2text = {num: text for text, num in text2num.items()}\n    train['num_less_toxic'] = train[

In [9]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [10]:
df_train.less_toxic = df_train.less_toxic.apply(text_cleaning)
df_train.more_toxic = df_train.more_toxic.apply(text_cleaning)
df_valid.less_toxic = df_valid.less_toxic.apply(text_cleaning)
df_valid.more_toxic = df_valid.more_toxic.apply(text_cleaning)

In [11]:
df_train = df_train.sample(frac=1, random_state=CONFIG['seed'])
df_valid = df_valid.sample(frac=1, random_state=CONFIG['seed'])

In [12]:
#df = get_group_unionfind(df)

In [13]:
'''group_kfold = GroupKFold(n_splits=5)
for fold, (trn_idx, val_idx) in enumerate(group_kfold.split(df, df, df['group'])): 
    df.loc[val_idx , "kfold"] = fold

df["kfold"] = df["kfold"].astype(int)
df.to_csv('train_noleak.csv', index=False)
df.head()'''

'group_kfold = GroupKFold(n_splits=5)\nfor fold, (trn_idx, val_idx) in enumerate(group_kfold.split(df, df, df[\'group\'])): \n    df.loc[val_idx , "kfold"] = fold\n\ndf["kfold"] = df["kfold"].astype(int)\ndf.to_csv(\'train_noleak.csv\', index=False)\ndf.head()'

In [14]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.int),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.int),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.int),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.int),
            'target': torch.tensor(target, dtype=torch.int)
        }


In [15]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):
        out = self.model(input_ids=ids,attention_mask=mask,
                             output_hidden_states=False)
        out = self.drop(out[1])
        out = out[1]
        outputs = self.fc(out)
        
        return outputs

In [16]:
def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)

In [17]:
'''class SoftLoss(nn.Module):
    def __init__(self):
        super(SoftLoss, self).__init__()
        
    def forward(self, pred1, pred2, target, margin = 0.5):
        #return torch.log(1.0 + torch.exp(pred1 - pred2 + margin))
        return nn.Softplus()(pred1 - pred2 + margin)'''

'class SoftLoss(nn.Module):\n    def __init__(self):\n        super(SoftLoss, self).__init__()\n        \n    def forward(self, pred1, pred2, target, margin = 0.5):\n        #return torch.log(1.0 + torch.exp(pred1 - pred2 + margin))\n        return nn.Softplus()(pred1 - pred2 + margin)'

In [18]:
'''def criterion(outputs1, outputs2, targets):
    return SoftLoss()(outputs1, outputs2, targets, CONFIG['margin'])'''

"def criterion(outputs1, outputs2, targets):\n    return SoftLoss()(outputs1, outputs2, targets, CONFIG['margin'])"

In [19]:
def accuracy(y_more_toxic, y_less_toxic): 
    correct = torch.sum(torch.where(y_more_toxic>y_less_toxic, 1, 0))
    wrong = torch.sum(torch.where(y_more_toxic<=y_less_toxic, 1, 0))
    acc = correct / (correct + wrong)
    return acc.view(-1).cpu().detach().numpy()

In [20]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    #Automatic Mixed Precision
    scaler = torch.cuda.amp.GradScaler()
    #------------------
    
    score_batch = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
            
        
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.int)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.int)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.int)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.int)
        targets = data['target'].to(device, dtype=torch.int)

        batch_size = more_toxic_ids.size(0)

        #Automatic Mixed Precision
        with torch.cuda.amp.autocast():
        #------------------
            more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
            less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
            loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
            loss = loss / CONFIG['n_accumulate']
            
            score_batch.append(accuracy(more_toxic_outputs, less_toxic_outputs))
        
        #Automatic Mixed Precision
        scaler.scale(loss).backward()
        #------------------

        if (step + 1) % CONFIG['n_accumulate'] == 0:
                
            #optimizer.step()
            
            #Automatic Mixed Precision
            scaler.step(optimizer)
            scaler.update()
            #------------------
            
            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

 
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        
        epoch_score = np.mean(score_batch)

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    return epoch_loss, epoch_score


In [21]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    score_batch = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.int)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.int)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.int)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.int)
        targets = data['target'].to(device, dtype=torch.int)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        score_batch.append(accuracy(more_toxic_outputs, less_toxic_outputs))
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        epoch_score = np.mean(score_batch)
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    return epoch_loss, epoch_score


In [22]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    epochs_no_improve = 0
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_score = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train Score'].append(train_epoch_score)
        history['Valid Score'].append(val_epoch_score)
        
        print(f"{b_}Train score {train_epoch_score}")
        print(f"{b_}Valid score {val_epoch_score}")
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            history["Best Loss"].append(best_epoch_loss)
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= CONFIG["patience"]:
                print('Early stopping!' )
                print()
                break
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [23]:
def prepare_loaders(fold):
    #df_train = df[df.kfold != fold].reset_index(drop=True)
    #df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = JigsawDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = JigsawDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [24]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [25]:
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = JigsawModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], eps=CONFIG['epsilon'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()
    

====== Fold: 0 ======


Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: NVIDIA GeForce RTX 2080



100%|█| 2147/2147 [10:16<00:00,  3.48it/s, Epoch=1, LR=8.11e-6, Train_Loss=0.437
100%|██| 538/538 [02:59<00:00,  3.00it/s, Epoch=1, LR=8.11e-6, Valid_Loss=0.454]


Train score 0.5961807370185852
Valid score 0.6505576372146606
Validation Loss Improved (inf ---> 0.45401408647448005)
Model Saved



...

Train score 0.6478807926177979
Valid score 0.6505576372146606




100%|█| 2147/2147 [10:15<00:00,  3.49it/s, Epoch=4, LR=1.21e-6, Train_Loss=0.395
100%|██| 538/538 [02:57<00:00,  3.03it/s, Epoch=4, LR=1.21e-6, Valid_Loss=0.472]

Train score 0.6422915458679199
Valid score 0.6375464797019958




100%|███| 538/538 [02:58<00:00,  3.02it/s, Epoch=5, LR=4.8e-6, Valid_Loss=0.502]

Train score 0.6627852916717529
Valid score 0.6486988663673401
Early stopping!

Training complete in 1h 6m 17s
Best Loss: 0.4540

